## Importação do dataset

In [1]:
pip install transformers datasets evaluate accelerate --user

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset

datasetTrain = load_dataset("md_gender_bias", "opensubtitles_inferred", split="train")
datasetTest = load_dataset("md_gender_bias", "opensubtitles_inferred", split="test")

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the module from C:\Users\mclar\.cache\huggingface\modules\datasets_modules\datasets\md_gender_bias\8ae77b51acf93383161cc954b146159291beca6c979b54ce228c46db86116c05 (last modified on Thu May  2 14:28:43 2024) since it couldn't be found locally at md_gender_bias, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from C:\Users\mclar\.cache\huggingface\modules\datasets_modules\datasets\md_gender_bias\8ae77b51acf93383161cc954b146159291beca6c979b54ce228c46db86116c05 (last modified on Thu May  2 14:28:43 2024) since it couldn't be found locally at md_gender_bias, or remotely on the Hugging Face Hub.


## Pré-processamento

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

In [4]:
def preprocess_function(examples):
    label_column = 'ternary_label'
    return {'input_ids': tokenizer(examples['text'], truncation=True)['input_ids'],
            'labels': examples[label_column]}

In [5]:
tokenized_datasetTrain = datasetTrain.map(preprocess_function, batched=True)
tokenized_datasetTest = datasetTest.map(preprocess_function, batched=True)

In [6]:
tokenized_datasetTrain

Dataset({
    features: ['text', 'binary_label', 'binary_score', 'ternary_label', 'ternary_score', 'input_ids', 'labels'],
    num_rows: 351036
})

In [7]:
tokenized_datasetTest

Dataset({
    features: ['text', 'binary_label', 'binary_score', 'ternary_label', 'ternary_score', 'input_ids', 'labels'],
    num_rows: 49108
})

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluate

In [9]:
pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import evaluate

accuracy = evaluate.load("accuracy")

In [11]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Train

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [13]:
id2label = {0: "ABOUT:female", 1: "ABOUT:male", 2: "ABOUT:gender-neutral"}
label2id = {"ABOUT:female": 0, "ABOUT:male": 1, "ABOUT:gender-neutral": 2}

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
training_args = TrainingArguments(
    output_dir="mdgenderbias_bert",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasetTrain,
    eval_dataset=tokenized_datasetTest,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\mclar\AppData\Roaming\Python\Python312\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  1%|          | 500/43880 [41:08<55:34:32,  4.61s/it]

{'loss': 0.632, 'grad_norm': 7.899115562438965, 'learning_rate': 1.977210574293528e-05, 'epoch': 0.02}


  2%|▏         | 1000/43880 [1:23:59<59:48:15,  5.02s/it]

{'loss': 0.4554, 'grad_norm': 8.819112777709961, 'learning_rate': 1.9544211485870555e-05, 'epoch': 0.05}


  3%|▎         | 1500/43880 [2:06:50<59:14:54,  5.03s/it]

...

                                                        
100%|██████████| 43880/43880 [64:07:16<00:00,  2.85s/it]

{'eval_loss': 0.3602883219718933, 'eval_accuracy': 0.8996701148489045, 'eval_runtime': 2065.1091, 'eval_samples_per_second': 23.78, 'eval_steps_per_second': 1.487, 'epoch': 2.0}


100%|██████████| 43880/43880 [64:07:19<00:00,  5.26s/it]

{'train_runtime': 230839.8497, 'train_samples_per_second': 3.041, 'train_steps_per_second': 0.19, 'train_loss': 0.2574123617292211, 'epoch': 2.0}


TrainOutput(global_step=43880, training_loss=0.2574123617292211, metrics={'train_runtime': 230839.8497, 'train_samples_per_second': 3.041, 'train_steps_per_second': 0.19, 'train_loss': 0.2574123617292211, 'epoch': 2.0})

In [16]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: sentencepiece in c:\python312\lib\site-packages (0.2.0)




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from transformers import AutoTokenizer, BertTokenizer, BertForSequenceClassification, BertConfig

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

config = BertConfig.from_pretrained("mdgenderbias_bert/checkpoint-43880/config.json")

model = BertForSequenceClassification.from_pretrained(
    "mdgenderbias_bert/checkpoint-43880/model.safetensors", 
    config=config
)

In [5]:
from datasets import load_dataset

datasetValidation = load_dataset("md_gender_bias", "opensubtitles_inferred", split="validation")
datasetValidation

Using the latest cached version of the module from C:\Users\mclar\.cache\huggingface\modules\datasets_modules\datasets\md_gender_bias\8ae77b51acf93383161cc954b146159291beca6c979b54ce228c46db86116c05 (last modified on Thu May  2 14:28:43 2024) since it couldn't be found locally at md_gender_bias, or remotely on the Hugging Face Hub.


Dataset({
    features: ['text', 'binary_label', 'binary_score', 'ternary_label', 'ternary_score'],
    num_rows: 41957
})

In [19]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import torch

def fazer_previsao(texto):
    inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    previsao = torch.argmax(outputs.logits).item()
    return previsao


previsoes = { 'text': [], 'prediction': []}

for exemplo in datasetValidation['text']:
    previsao = fazer_previsao(exemplo)
    previsoes['text'].append(exemplo)
    previsoes['prediction'].append(previsao)


df = pd.DataFrame(previsoes)

datatoexcel = pd.ExcelWriter('validation_prediction-bert.xlsx')

df.to_excel(datatoexcel)

datatoexcel.close()

In [7]:
cont = 0

for pred, label in zip(previsoes['prediction'], datasetValidation['ternary_label']):
    if pred == label:
        cont += 1

print(len(datasetValidation['ternary_label']), (cont/len(datasetValidation['ternary_label']))*100)

41957 89.83244750577973


In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

y_pred = pd.read_excel("validation_prediction-bert.xlsx")
y_pred = y_pred['prediction']
y_true = datasetValidation['ternary_label']

precision = precision_score(y_true, y_pred, average='weighted')
print("Precisão:", precision)

recall = recall_score(y_true, y_pred, average='weighted')
print("Revocação:", recall)

f1 = f1_score(y_true, y_pred, average='weighted')
print("F1-score:", f1)

Precisão: 0.8984636141616736
Revocação: 0.8983244750577972
F1-score: 0.8983148667006122
